<!--BOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="figures/PDSH-cover-small.png">

*このノートブックには、Jake VanderPlas による [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do) からの抜粋が含まれています。コンテンツは利用可能です [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do).*

※テキストは[CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode)で、コードは[CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode)で公開しています。このコンテンツが役立つと思われる場合は、[CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode) による作業のサポートを検討してください!*

<!--ナビゲーション-->
< [Combining Datasets: Merge and Join](03.07-Merge-and-Join.ipynb) | [Combining Datasets: Merge and Join](03.07-Merge-and-Join.ipynb) | [Combining Datasets: Merge and Join](03.07-Merge-and-Join.ipynb) >

<a href="https://colab.research.google.com/github/vitroid/PythonDataScienceHandbook/blob/ja/notebooks/03.08-Aggregation-and-Grouping.ipynb"><img align="left" src=" https://colab.research.google.com/assets/colab-badge.svg" alt="Colab で開く" title="Google Colaboratory で開いて実行する"></a>


# 集計とグループ化

大規模なデータの分析に不可欠な部分は、効率的な要約です: ``sum()``、``mean()``、``median()``、``min()``、および `` max()`` では、単一の数値が、潜在的に大きなデータセットの性質についての洞察を与えます。
このセクションでは、NumPy 配列で見たような単純な操作から、groupby の概念に基づくより洗練された操作まで、Pandas の集計について説明します。

便宜上、前のセクションで見たのと同じ ``display`` マジック関数を使用します:

In [1]:
import numpy as np
import pandas as pd

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

## 惑星データ

ここでは、[Seaborn package](http://seaborn.pydata.org/) から入手できる Planets データセットを使用します ([Seaborn package](http://seaborn.pydata.org/) を参照)。
これは、天文学者が他の星の周りで発見した惑星に関する情報を提供します (*太陽系外惑星* または略して *太陽系外惑星* として知られています)。簡単な Seaborn コマンドでダウンロードできます。

In [2]:
import seaborn as sns
planets = sns.load_dataset('planets')
planets.shape

(1035, 6)

In [3]:
planets.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


これには、2014 年までに発見された 1,000 以上の太陽系外惑星に関する詳細が含まれています。

## Pandas での単純な集計

前に、NumPy 配列 (["Aggregations: Min, Max, and Everything In Between"](02.04-Computation-on-arrays-aggregates.ipynb)) で使用できるデータ集計のいくつかを調べました。
1 次元の NumPy 配列と同様に、Pandas ``Series`` の場合、集計は単一の値を返します。

In [4]:
rng = np.random.RandomState(42)
ser = pd.Series(rng.rand(5))
ser

0    0.374540
1    0.950714
2    0.731994
3    0.598658
4    0.156019
dtype: float64

In [5]:
ser.sum()

2.8119254917081569

In [6]:
ser.mean()

0.56238509834163142

``DataFrame`` の場合、デフォルトでは、集計は各列内で結果を返します:

In [7]:
df = pd.DataFrame({'A': rng.rand(5),
                   'B': rng.rand(5)})
df

,A,B
0,0.155995,0.020584
1,0.058084,0.969910
2,0.866176,0.832443
3,0.601115,0.212339
4,0.708073,0.181825


In [8]:
df.mean()

A    0.477888
B    0.443420
dtype: float64

``axis`` 引数を指定することで、代わりに各行内で集計できます:

In [9]:
df.mean(axis='columns')

0    0.088290
1    0.513997
2    0.849309
3    0.406727
4    0.444949
dtype: float64

Pandas ``Series`` および ``DataFrame`` には、[Aggregations: Min, Max, and Everything In Between](02.04-Computation-on-arrays-aggregates.ipynb) で言及されている共通の集約がすべて含まれています。さらに、各列に対していくつかの一般的な集計を計算し、結果を返す便利なメソッド ``describe()`` があります。
これを Planets データで使用してみましょう。今のところ、値が欠落している行を削除します。

In [10]:
planets.dropna().describe()

,number,orbital_period,mass,distance,year
count,498.00000,498.000000,498.000000,498.000000,498.000000
mean,1.73494,835.778671,2.509320,52.068213,2007.377510
std,1.17572,1469.128259,3.636274,46.596041,4.167284
min,1.00000,1.328300,0.003600,1.350000,1989.000000
25%,1.00000,38.272250,0.212500,24.497500,2005.000000
50%,1.00000,357.000000,1.245000,39.940000,2009.000000
75%,2.00000,999.600000,2.867500,59.332500,2011.000000
max,6.00000,17337.500000,25.000000,354.000000,2014.000000


これは、データセットの全体的なプロパティを理解し始めるのに役立つ方法です。
たとえば、「年」の列を見ると、太陽系外惑星は 1989 年まで発見されていましたが、知られているすべてのエクスポラネットの半分は 2010 年以降まで発見されていませんでした。
これは主に、他の星の周りで日食を起こしている惑星を見つけるために特別に設計された宇宙ベースの望遠鏡である *ケプラー* ミッションのおかげです。

次の表は、その他の組み込みの Pandas 集計をまとめたものです。

| |集計 |説明 |
|--------------------------|---------------------------------|
| | ``count()`` |アイテムの総数 |
| | ``first()``, ``last()`` |最初で最後のアイテム |
| | ``mean()``, ``median()`` |平均と中央値 |
| | ``min()``, ``max()`` |最小値と最大値 |
| | ``std()``, ``var()`` |標準偏差と分散 |
| | ``mad()`` |平均絶対偏差 |
| | ``prod()`` |全商品の商品 |
| | ``sum()`` |すべての項目の合計 |

これらはすべて ``DataFrame`` および ``Series`` オブジェクトのメソッドです。

ただし、データをさらに深く掘り下げるには、単純な集計では不十分なことがよくあります。
データ要約の次のレベルは ``groupby`` 操作です。これにより、データのサブセットの集計を迅速かつ効率的に計算できます。

## GroupBy: 分割、適用、結合

単純な集計はデータセットのフレーバーを与えることができますが、多くの場合、いくつかのラベルまたはインデックスで条件付きで集計することを好みます: これは、いわゆる ``groupby`` 操作で実装されます。
「グループ化」という名前は、SQL データベース言語のコマンドに由来しますが、Rstats で有名な Hadley Wickham によって最初に造られた用語である *split、apply、combine* で考えたほうがおそらくわかりやすいでしょう。

### 分割、適用、結合

「適用」が合計集計である、この分割-適用-結合操作の標準的な例を次の図に示します。

![](figures/03.08-split-apply-combine.png)
[figure source in Appendix](06.00-Figure-Code.ipynb#Split-Apply-Combine)

これにより、「groupby」が何を達成するかが明確になります。

- *分割* ステップでは、指定されたキーの値に応じて ``DataFrame`` を分割およびグループ化します。
- *適用* ステップには、個々のグループ内で、通常は集計、変換、またはフィルタリングなどの関数を計算することが含まれます。
- *combine* ステップは、これらの操作の結果を出力配列にマージします。

これは、以前に説明したマスキング、集約、およびマージ コマンドの組み合わせを使用して手動で行うことができますが、重要なことは、*中間分割を明示的にインスタンス化する必要がない*ことです。むしろ ``GroupBy`` は (多くの場合) データの 1 回のパスでこれを行うことができ、途中で各グループの合計、平均、カウント、最小、またはその他の集計を更新します。
``GroupBy`` の力は、これらのステップを抽象化することです: ユーザーは、内部で計算が *どのように行われるかを考える必要はなく、*全体としての操作* について考える必要があります。

具体的な例として、この図に示されている計算に Pandas を使用することを見てみましょう。
入力 ``DataFrame`` を作成することから始めます。

In [11]:
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data': range(6)}, columns=['key', 'data'])
df

,key,data
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


最も基本的な split-apply-combine 操作は、必要なキー列の名前を渡して、DataFrame の groupby() メソッドで計算できます。

In [12]:
df.groupby('key')

返されるのは ``DataFrame`` のセットではなく、 ``DataFrameGroupBy`` オブジェクトであることに注意してください。
このオブジェクトは魔法の場所です: ``DataFrame`` の特別なビューと考えることができます。これは、グループを掘り下げる準備ができていますが、集計が適用されるまで実際の計算は行いません。
この「遅延評価」アプローチは、一般的な集計を、ユーザーに対してほぼ透過的な方法で非常に効率的に実装できることを意味します。

結果を生成するために、この ``DataFrameGroupBy`` オブジェクトに集計を適用できます。これにより、適切な適用/結合手順が実行され、目的の結果が生成されます。

In [13]:
df.groupby('key').sum()

,data
key,
A,3
B,5
C,7


``sum()`` メソッドは、ここでの 1 つの可能性にすぎません。次の説明で説明するように、実質的にすべての一般的な Pandas または NumPy 集計関数と、実質的にすべての有効な ``DataFrame`` 操作を適用できます。

### GroupBy オブジェクト

``GroupBy`` オブジェクトは非常に柔軟な抽象化です。
多くの点で、それをあたかも ``DataFrame`` のコレクションであるかのように簡単に扱うことができ、内部で難しいことを実行します。 Planets データを使用した例をいくつか見てみましょう。

おそらく、GroupBy によって利用可能になる最も重要な操作は、*aggregate*、*filter*、*transform*、および *apply* です。
これらのそれぞれについては、["Aggregate, Filter, Transform, Apply"](#Aggregate,-Filter,-Transform,-Apply) で詳しく説明しますが、その前に、基本的な ``GroupBy`` 操作で使用できる他の機能をいくつか紹介しましょう。

#### 列の索引付け

``GroupBy`` オブジェクトは ``DataFrame`` と同じ方法で列のインデックス付けをサポートし、修正された ``GroupBy`` オブジェクトを返します。
例えば：

In [14]:
planets.groupby('method')

In [15]:
planets.groupby('method')['orbital_period']

ここでは、列名を参照して、元の ``DataFrame`` グループから特定の ``Series`` グループを選択しました。
``GroupBy`` オブジェクトと同様に、オブジェクトの集計を呼び出すまで計算は行われません:

In [16]:
planets.groupby('method')['orbital_period'].median()

method
Astrometry                         631.180000
Eclipse Timing Variations         4343.500000
Imaging                          27500.000000
Microlensing                      3300.000000
Orbital Brightness Modulation        0.342887
Pulsar Timing                       66.541900
Pulsation Timing Variations       1170.000000
Radial Velocity                    360.200000
Transit                              5.714932
Transit Timing Variations           57.011000
Name: orbital_period, dtype: float64

これにより、各方法が敏感な軌道周期 (日単位) の一般的なスケールがわかります。

#### グループの繰り返し

``GroupBy`` オブジェクトは、各グループを ``Series`` または ``DataFrame`` として返す、グループに対する直接の反復をサポートします。

In [17]:
for (method, group) in planets.groupby('method'):
    print("{0:30s} shape={1}".format(method, group.shape))

Astrometry                     shape=(2, 6)
Eclipse Timing Variations      shape=(9, 6)
Imaging                        shape=(38, 6)
Microlensing                   shape=(23, 6)
Orbital Brightness Modulation  shape=(3, 6)
Pulsar Timing                  shape=(5, 6)
Pulsation Timing Variations    shape=(1, 6)
Radial Velocity                shape=(553, 6)
Transit                        shape=(397, 6)
Transit Timing Variations      shape=(4, 6)


これは、特定のことを手動で行うのに役立ちますが、組み込みの「適用」機能を使用する方がはるかに高速であることがよくあります。これについては後で説明します。

#### 発送方法

Python のクラス マジックによって、GroupBy オブジェクトによって明示的に実装されていないメソッドはすべて通過され、グループ上で呼び出されます。
たとえば、DataFrame の describe() メソッドを使用して、データ内の各グループを説明する一連の集計を実行できます。

In [18]:
planets.groupby('method')['year'].describe().unstack()

,count,mean,std,min,25%,50%,75%,max
method,,,,,,,,
Astrometry,2.0,2011.500000,2.121320,2010.0,2010.75,2011.5,2012.25,2013.0
Eclipse Timing Variations,9.0,2010.000000,1.414214,2008.0,2009.00,2010.0,2011.00,2012.0
Imaging,38.0,2009.131579,2.781901,2004.0,2008.00,2009.0,2011.00,2013.0
Microlensing,23.0,2009.782609,2.859697,2004.0,2008.00,2010.0,2012.00,2013.0
Orbital Brightness Modulation,3.0,2011.666667,1.154701,2011.0,2011.00,2011.0,2012.00,2013.0
Pulsar Timing,5.0,1998.400000,8.384510,1992.0,1992.00,1994.0,2003.00,2011.0
Pulsation Timing Variations,1.0,2007.000000,NaN,2007.0,2007.00,2007.0,2007.00,2007.0
Radial Velocity,553.0,2007.518987,4.249052,1989.0,2005.00,2009.0,2011.00,2014.0
Transit,397.0,2011.236776,2.077867,2002.0,2010.00,2012.0,2013.00,2014.0


この表を見ると、データをよりよく理解するのに役立ちます。たとえば、大部分の惑星は動径速度法とトランジット法によって発見されていますが、後者は (新しい、より正確な望遠鏡のおかげで) 過去 10 年間で一般的になっただけです。 .
最新の方法は、2011 年まで新しい惑星を発見するために使用されなかったトランジット タイミング変動と軌道輝度変調のようです。

これは、dispatch メソッドの有用性のほんの一例です。
これらは*個々のグループ*に適用され、結果は ``GroupBy`` 内で結合されて返されることに注意してください。
繰り返しになりますが、有効な ``DataFrame``/``Series`` メソッドは、対応する ``GroupBy`` オブジェクトで使用できます。これにより、非常に柔軟で強力な操作が可能になります!

### 集計、フィルター、変換、適用

これまでの説明では、結合操作の集計に焦点を当てていましたが、他にも使用可能なオプションがあります。
特に、GroupBy オブジェクトには、さまざまな便利な機能を効率的に実装する ``aggregate()``、``filter()``、``transform()``、および ``apply()`` メソッドがあります。グループ化されたデータを結合する前の操作。

以下のサブセクションでは、この ``DataFrame`` を使用します。

In [19]:
rng = np.random.RandomState(0)
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data1': range(6),
                   'data2': rng.randint(0, 10, 6)},
                   columns = ['key', 'data1', 'data2'])
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


#### 集計

``sum()`` や ``median()`` などを使った ``GroupBy`` 集約に慣れてきましたが、 ``aggregate()`` メソッドはさらに柔軟です。
文字列、関数、またはそれらのリストを取り、すべての集計を一度に計算できます。
これらすべてを組み合わせた簡単な例を次に示します。

In [20]:
df.groupby('key').aggregate(['min', np.median, max])

data1            data2           
      min median max   min median max
key                                  
A       0    1.5   3     3    4.0   5
B       1    2.5   4     0    3.5   7
C       2    3.5   5     3    6.0   9

別の便利なパターンは、列名をその列に適用される操作にマッピングする辞書を渡すことです。

In [21]:
df.groupby('key').aggregate({'data1': 'min',
                             'data2': 'max'})

,data1,data2
key,,
A,0,5
B,1,7
C,2,9


#### フィルタリング

フィルタリング操作により、グループ プロパティに基づいてデータを削除できます。
たとえば、標準偏差がある臨界値よりも大きいすべてのグループを保持したい場合があります。

In [22]:
def filter_func(x):
    return x['data2'].std() > 4

display('df', "df.groupby('key').std()", "df.groupby('key').filter(filter_func)")

df
  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

df.groupby('key').std()
       data1     data2
key                   
A    2.12132  1.414214
B    2.12132  4.949747
C    2.12132  4.242641

df.groupby('key').filter(filter_func)
  key  data1  data2
1   B      1      0
2   C      2      3
4   B      4      7
5   C      5      9

フィルター関数は、グループがフィルタリングを通過するかどうかを指定するブール値を返す必要があります。ここで、グループ A は標準偏差が 4 を超えていないため、結果から除外されます。

＃＃＃＃ 変身

集計では縮小されたバージョンのデータが返される必要がありますが、変換では完全なデータの一部の変換されたバージョンが返され、再結合されます。
このような変換の場合、出力は入力と同じ形になります。
一般的な例は、グループごとの平均を差し引いてデータを中央に配置することです。

In [23]:
df.groupby('key').transform(lambda x: x - x.mean())

,data1,data2
0,-1.5,1.0
1,-1.5,-3.5
2,-1.5,-3.0
3,1.5,-1.0
4,1.5,3.5
5,1.5,3.0


#### apply() メソッド

``apply()`` メソッドを使用すると、グループの結果に任意の関数を適用できます。
この関数は ``DataFrame`` を受け取り、Pandas オブジェクト (``DataFrame``、``Series`` など) またはスカラーを返す必要があります。結合操作は、返される出力のタイプに合わせて調整されます。

たとえば、最初の列を 2 番目の列の合計で正規化する ``apply()`` は次のとおりです。

In [24]:
def norm_by_data2(x):
    # x is a DataFrame of group values
    x['data1'] /= x['data2'].sum()
    return x

display('df', "df.groupby('key').apply(norm_by_data2)")

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9
,key,data1,data2
0,A,0.000000,5
1,B,0.142857,0
2,C,0.166667,3


``GroupBy`` 内の ``apply()`` は非常に柔軟です。唯一の基準は、関数が ``DataFrame`` を取り、Pandas オブジェクトまたはスカラーを返すことです。途中で何をするかはあなた次第です！

### 分割キーの指定

前に示した簡単な例では、単一の列名で ``DataFrame`` を分割しました。
これは、グループを定義できる多くのオプションの 1 つにすぎません。ここでは、グループを指定するためのその他のオプションについて説明します。

#### グループ化キーを提供するリスト、配列、シリーズ、またはインデックス

キーは ``DataFrame`` の長さと一致する任意のシリーズまたはリストにすることができます。例えば：

In [25]:
L = [0, 1, 0, 1, 2, 0]
display('df', 'df.groupby(L).sum()')

df
  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

df.groupby(L).sum()
   data1  data2
0      7     17
1      4      3
2      4      7

もちろん、これは以前の ``df.groupby('key')`` を達成する別の、より冗長な方法があることを意味します:

In [26]:
display('df', "df.groupby(df['key']).sum()")

df
  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

df.groupby(df['key']).sum()
     data1  data2
key              
A        3      8
B        5      7
C        7     12

#### グループへのディクショナリまたはシリーズ マッピング インデックス

もう 1 つの方法は、インデックス値をグループ キーにマップする辞書を提供することです。

In [27]:
df2 = df.set_index('key')
mapping = {'A': 'vowel', 'B': 'consonant', 'C': 'consonant'}
display('df2', 'df2.groupby(mapping).sum()')

,data1,data2
key,,
A,0,5
B,1,0
C,2,3
A,3,3
B,4,7
C,5,9
,data1,data2
consonant,12,19
vowel,3,8


#### 任意の Python 関数

マッピングと同様に、インデックス値を入力してグループを出力する任意の Python 関数を渡すことができます。

In [28]:
display('df2', 'df2.groupby(str.lower).mean()')

,data1,data2
key,,
A,0,5
B,1,0
C,2,3
A,3,3
B,4,7
C,5,9
,data1,data2
a,1.5,4.0
b,2.5,3.5


#### 有効なキーのリスト

さらに、前述のキーの選択肢のいずれかを組み合わせて、マルチインデックスでグループ化できます。

In [29]:
df2.groupby([str.lower, mapping]).mean()

,,data1,data2
a,vowel,1.5,4.0
b,consonant,2.5,3.5
c,consonant,3.5,6.0


### グループ化の例

この例として、数行の Python コードで、これらすべてをまとめて、発見された惑星をメソッドごとおよび 10 年ごとに数えることができます。

In [30]:
decade = 10 * (planets['year'] // 10)
decade = decade.astype(str) + 's'
decade.name = 'decade'
planets.groupby(['method', decade])['number'].sum().unstack().fillna(0)

decade,1980s,1990s,2000s,2010s
method,,,,
Astrometry,0.0,0.0,0.0,2.0
Eclipse Timing Variations,0.0,0.0,5.0,10.0
Imaging,0.0,0.0,29.0,21.0
Microlensing,0.0,0.0,12.0,15.0
Orbital Brightness Modulation,0.0,0.0,0.0,5.0
Pulsar Timing,0.0,9.0,1.0,1.0
Pulsation Timing Variations,0.0,0.0,1.0,0.0
Radial Velocity,1.0,52.0,475.0,424.0
Transit,0.0,0.0,64.0,712.0


これは、現実的なデータセットを見ると、これまで説明してきた操作の多くを組み合わせることの威力を示しています。
過去数十年間に惑星がいつ、どのように発見されたかについて、大まかな理解がすぐに得られます。

ここでは、これらの数行のコードを掘り下げ、個々のステップを評価して、結果に対して何を行っているかを正確に理解することをお勧めします。
確かにやや複雑な例ですが、これらの部分を理解することで、独自のデータを同様に探索する手段が得られます。

<!--ナビゲーション-->
< [Combining Datasets: Merge and Join](03.07-Merge-and-Join.ipynb) | [Combining Datasets: Merge and Join](03.07-Merge-and-Join.ipynb) | [Combining Datasets: Merge and Join](03.07-Merge-and-Join.ipynb) >

<a href="https://colab.research.google.com/github/vitroid/PythonDataScienceHandbook/blob/ja/notebooks/03.08-Aggregation-and-Grouping.ipynb"><img align="left" src=" https://colab.research.google.com/assets/colab-badge.svg" alt="Colab で開く" title="Google Colaboratory で開いて実行する"></a>
